## Instalacion e importacion de dependencias 

In [2]:
!pip install mediapipe opencv-python

     --------------------------------------- 50.3/50.3 MB 10.6 MB/s eta 0:00:00
     --------------------------------------- 38.1/38.1 MB 11.5 MB/s eta 0:00:00
  Using cached protobuf-3.20.3-cp310-cp310-win_amd64.whl (904 kB)
     ---------------------------------------- 199.7/199.7 KB ? eta 0:00:00
     --------------------------------------- 44.8/44.8 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.3
    Uninstalling protobuf-4.23.3:
      Successfully uninstalled protobuf-4.23.3


You should consider upgrading via the 'C:\Users\lucas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import cv2
import mediapipe as mp 
import numpy as np

mp_drawing = mp.solutions.drawing_utils

mp_pose = mp.solutions.pose  #pose estimation model 

In [4]:
# VIDEO FEED 
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow("Mediapipe Feed", frame)

    if cv2.waitKey(10) & 0XFF == ord("q"):
        break

cap.release()

cv2.destroyAllWindows()

## Make Detections

In [6]:
cap = cv2.VideoCapture(0)

#set up mediapipe instance 
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # make detection 
        results = pose.process(image)

        # recolor back to BGR
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Render detections 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        

        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(10) & 0XFF == ord("q"):
            break

cap.release()

cv2.destroyAllWindows()

 ## Determining Joints 

<img src="imgs/joints.png">

In [ ]:
cap = cv2.VideoCapture(0)

#set up mediapipe instance 
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # make detection 
        results = pose.process(image)

        # recolor back to BGR
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks 
        try:
            landmarks = results.pose_landmarks.landmark 

            print(landmarks)
        except:
            pass

        #Render detections 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        

        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(10) & 0XFF == ord("q"):
            break

cap.release()

cv2.destroyAllWindows()